<a href="https://colab.research.google.com/github/viswambaran/NUCapstone/blob/main/EPA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tagging news headlines using Zero Shot Classification - BART model

In [ ]:
!python -V

Python 3.10.12


In [ ]:
!pip install transformers
!pip install NLTK

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.4 MB/s eta 0:00:00


In [ ]:
import re
import textwrap

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
from transformers import BartForSequenceClassification, BartTokenizer, pipeline

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:


financial_news_taxonomy = {
    "Market Movements": ["Stock Trends", "Commodity Prices", "Currency Rates", "Bond Yields"],
    "Company News": ["Earnings", "Product Launches", "M&A", "Legal & Regulatory", "Leadership"],
    "Economic Data": ["Employment", "Inflation", "GDP", "Consumer Sentiment"],
    "Policy & Regulation": ["Central Bank Actions", "Trade & Tax Policies", "Environmental Rules"],
    "Global Events": ["Geopolitical Issues", "Elections", "Natural Disasters", "Health Crises"],
    "Sector Highlights": ["Tech Developments", "Banking News", "Energy Updates", "Healthcare Innovations", "Real Estate Trends"],
    "Investment Insights": ["Fund Activities", "Asset Trends", "Investment Strategies"]
}

financial_bow = [
    "stocks", "shares", "dividend", "portfolio", "equity", "bonds", "securities",
    "exchange", "interest", "rate", "yield", "return on investment", "ROI",
    "capital", "asset", "liability", "balance sheet", "income statement",
    "cash flow", "debt", "credit", "loan", "mortgage", "collateral",
    "bankruptcy", "merger", "acquisition", "IPO", "initial public offering",
    "commodities", "futures", "options", "hedge", "fund", "mutual fund",
    "index", "bull market", "bear market", "recession", "economy",
    "inflation", "deflation", "GDP", "gross domestic product", "valuation",
    "earnings", "P/E ratio", "profit", "loss", "revenue", "expense",
    "tax", "tariff", "divest", "stake", "capital gain", "liquidity",
    "fiduciary", "derivative", "leverage", "broker", "trader", "analyst",
    "rating", "outlook", "forecast", "volatile", "risk", "benchmark",
    "monetary policy", "central bank", "Federal Reserve", "currency", "forex",
    "exchange rate"
]

print(financial_bow)


print(financial_news_taxonomy)

financial_news_terms = [item for sublist in financial_news_taxonomy.values() for item in sublist]


['stocks', 'shares', 'dividend', 'portfolio', 'equity', 'bonds', 'securities', 'exchange', 'interest', 'rate', 'yield', 'return on investment', 'ROI', 'capital', 'asset', 'liability', 'balance sheet', 'income statement', 'cash flow', 'debt', 'credit', 'loan', 'mortgage', 'collateral', 'bankruptcy', 'merger', 'acquisition', 'IPO', 'initial public offering', 'commodities', 'futures', 'options', 'hedge', 'fund', 'mutual fund', 'index', 'bull market', 'bear market', 'recession', 'economy', 'inflation', 'deflation', 'GDP', 'gross domestic product', 'valuation', 'earnings', 'P/E ratio', 'profit', 'loss', 'revenue', 'expense', 'tax', 'tariff', 'divest', 'stake', 'capital gain', 'liquidity', 'fiduciary', 'derivative', 'leverage', 'broker', 'trader', 'analyst', 'rating', 'outlook', 'forecast', 'volatile', 'risk', 'benchmark', 'monetary policy', 'central bank', 'Federal Reserve', 'currency', 'forex', 'exchange rate']
{'Market Movements': ['Stock Trends', 'Commodity Prices', 'Currency Rates', 'Bo

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
device

device(type='cpu')

# Cleansing dataset

In [ ]:
#pd.read_csv("drive/MyDrive/EPA/dataset/analyst_ratings_processed.csv")

ParserError: ignored

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# Sample dataset loading
headlines_df = pd.read_csv("drive/MyDrive/EPA/dataset/analyst_ratings_processed.csv")


# Initialize lemmatizer and define English stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def process_headline(headline):
    # Tokenize
    tokens = word_tokenize(headline)

    # Remove stopwords and lemmatize
    clean_tokens = [lemmatizer.lemmatize(token) for token in tokens if token.lower() not in stop_words]

    return " ".join(clean_tokens)

# Applying the processing function to each headline
headlines_df['processed_headline'] = headlines_df['title'].apply(process_headline)


In [ ]:
import pandas as pd
import nltk
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Ensure that the required NLTK resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')

# Load the dataset (assuming it's been downloaded and extracted as "stock_data.csv")

# Tokenize the headlines
tokenized_headlines = headlines_df['title'].apply(word_tokenize)

# Flatten the list of tokens and remove stopwords
all_tokens = [token.lower() for headline in tokenized_headlines for token in headline]
filtered_tokens = [token for token in all_tokens if token not in stopwords.words('english') and token.isalpha()]

# Frequency Analysis
word_freq = Counter(filtered_tokens)
common_terms = word_freq.most_common(200)  # Adjust the number based on desired length

# Extracting the terms
financial_bag_of_words = [item[0] for item in common_terms]

print(financial_bag_of_words)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
headlines_df.head()

,Unnamed: 0,title,date,stock,processed_headline
0,0.0,Stocks That Hit 52-Week Highs On Friday,2020-06-05 10:30:00-04:00,A,Stocks Hit 52-Week Highs Friday
1,1.0,Stocks That Hit 52-Week Highs On Wednesday,2020-06-03 10:45:00-04:00,A,Stocks Hit 52-Week Highs Wednesday
2,2.0,71 Biggest Movers From Friday,2020-05-26 04:30:00-04:00,A,71 Biggest Movers Friday
3,3.0,46 Stocks Moving In Friday's Mid-Day Session,2020-05-22 12:45:00-04:00,A,46 Stocks Moving Friday 's Mid-Day Session
4,4.0,B of A Securities Maintains Neutral on Agilent...,2020-05-22 11:38:00-04:00,A,B Securities Maintains Neutral Agilent Technol...


In [ ]:
from transformers.models.bart.modeling_bart import BartForConditionalGeneration
model_name = "facebook/bart-large-mnli"
tokeniser = BartTokenizer.from_pretrained(model_name)
model = BartForSequenceClassification.from_pretrained(model_name)
model2 = BartForConditionalGeneration.from_pretrained(model_name)

classifier = pipeline(
    task="zero-shot-classification",
    model=model_name,
    tokenizer=tokeniser,
    device=device.index
)

# Initialize zero-shot classifier
#classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")


In [ ]:
def classify_headline(headline, labels):
    result = classifier(headline, labels)
    return result['labels'][0]

#headlines_df['predicted_label'] = headlines_df['title'].apply(lambda x: classify_headline(x, financial_bow))

In [ ]:
classify_headline("Stock growth in yearrs", financial_bow)

'stocks'

In [ ]:
shortend_df = headlines_df[0:10]

shortend_df['predicted_label'] = shortend_df['title'].apply(lambda x: classify_headline(x, financial_bow))

<ipython-input-16-db2c6684a40b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shortend_df['predicted_label'] = shortend_df['title'].apply(lambda x: classify_headline(x, financial_bow))


In [ ]:
shortend_df

,Unnamed: 0,title,date,stock,processed_headline,predicted_label
0,0.0,Stocks That Hit 52-Week Highs On Friday,2020-06-05 10:30:00-04:00,A,Stocks Hit 52-Week Highs Friday,stocks
1,1.0,Stocks That Hit 52-Week Highs On Wednesday,2020-06-03 10:45:00-04:00,A,Stocks Hit 52-Week Highs Wednesday,stocks
2,2.0,71 Biggest Movers From Friday,2020-05-26 04:30:00-04:00,A,71 Biggest Movers Friday,volatile
3,3.0,46 Stocks Moving In Friday's Mid-Day Session,2020-05-22 12:45:00-04:00,A,46 Stocks Moving Friday 's Mid-Day Session,stocks
4,4.0,B of A Securities Maintains Neutral on Agilent...,2020-05-22 11:38:00-04:00,A,B Securities Maintains Neutral Agilent Technol...,analyst
5,5.0,"CFRA Maintains Hold on Agilent Technologies, L...",2020-05-22 11:23:00-04:00,A,"CFRA Maintains Hold Agilent Technologies , Low...",analyst
6,6.0,"UBS Maintains Neutral on Agilent Technologies,...",2020-05-22 09:36:00-04:00,A,"UBS Maintains Neutral Agilent Technologies , R...",analyst
7,7.0,Agilent Technologies shares are trading higher...,2020-05-22 09:07:00-04:00,A,Agilent Technologies share trading higher comp...,shares
8,8.0,Wells Fargo Maintains Overweight on Agilent Te...,2020-05-22 08:37:00-04:00,A,Wells Fargo Maintains Overweight Agilent Techn...,analyst
9,9.0,10 Biggest Price Target Changes For Friday,2020-05-22 08:06:00-04:00,A,10 Biggest Price Target Changes Friday,volatile


In [ ]:
sequence_to_classify = "one day I will see the world"
candidate_labels = ['travel', 'cooking', 'dancing']
results= classifier("stock price increase for apple", financial_bow)

In [ ]:
def softmax_results(results:dict):
  _categories=zip(results['labels'], results['scores'])
  sorted_categories=sorted(_categories, key=lambda x:x[1], reverse=True)
  for category, score in sorted_categories[:3]:
    print(f"{category}: {score:.3f}")

In [ ]:
df = pd.read_csv("drive/MyDrive/EPA/dataset/analyst_ratings_processed.csv")

df1 = pd.read_csv("drive/MyDrive/EPA/dataset/raw_partner_headlines.csv")




In [ ]:
df2 = pd.read_csv("drive/MyDrive/EPA/dataset/raw_analyst_ratings.csv")

In [ ]:
df2.head()

,Unnamed: 0,headline,url,publisher,date,stock
0,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54-04:00,A
1,1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03 10:45:20-04:00,A
2,2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26 04:30:07-04:00,A
3,3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22 12:45:06-04:00,A
4,4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22 11:38:59-04:00,A


In [ ]:
df1.head()

,Unnamed: 0,headline,url,publisher,date,stock
0,2,Agilent Technologies Announces Pricing of $5……...,http://www.gurufocus.com/news/1153187/agilent-...,GuruFocus,2020-06-01 00:00:00,A
1,3,Agilent (A) Gears Up for Q2 Earnings: What's i...,http://www.zacks.com/stock/news/931205/agilent...,Zacks,2020-05-18 00:00:00,A
2,4,J.P. Morgan Asset Management Announces Liquida...,http://www.gurufocus.com/news/1138923/jp-morga...,GuruFocus,2020-05-15 00:00:00,A
3,5,"Pershing Square Capital Management, L.P. Buys ...",http://www.gurufocus.com/news/1138704/pershing...,GuruFocus,2020-05-15 00:00:00,A
4,6,Agilent Awards Trilogy Sciences with a Golden ...,http://www.gurufocus.com/news/1134012/agilent-...,GuruFocus,2020-05-12 00:00:00,A


In [ ]:


financial_news_terms

['Stock Trends',
 'Commodity Prices',
 'Currency Rates',
 'Bond Yields',
 'Earnings',
 'Product Launches',
 'M&A',
 'Legal & Regulatory',
 'Leadership',
 'Employment',
 'Inflation',
 'GDP',
 'Consumer Sentiment',
 'Central Bank Actions',
 'Trade & Tax Policies',
 'Environmental Rules',
 'Geopolitical Issues',
 'Elections',
 'Natural Disasters',
 'Health Crises',
 'Tech Developments',
 'Banking News',
 'Energy Updates',
 'Healthcare Innovations',
 'Real Estate Trends',
 'Fund Activities',
 'Asset Trends',
 'Investment Strategies']

In [ ]:
# Testing out zero shot
#labels = ['Credit Risk', 'Market Risk', 'Liquidity Risk', 'Operational Risk', 'Generic Risk']
sample = df['title'][1600]
results = classifier(sample, candidate_labels=financial_news_terms)

In [ ]:
df['title'][1600]

'Schawk Announces Second-Quarter and First-Six Months 2009 Results'

In [ ]:
softmax_results(results)

securities: 0.142
asset: 0.118
stake: 0.102


In [ ]:
# Flattening the risk_taxonomy dictionary
all_elements = [item for sublist in risk_taxonomy.values() for item in sublist]

print(all_elements)


['Default Events', 'Credit Downgrades', 'Collateral Value Changes', 'Counterparty Failures', 'Loan Delinquencies', 'Stock Market Declines', 'Interest Rate Hikes', 'Exchange Rate Fluctuations', 'Commodity Price Shocks', 'Volatility Spikes', 'Banking Panics', 'Asset Sell-offs', 'Credit Squeeze', 'Shortage in Funding Markets', 'Market Illiquidity Events', 'Data Breaches', 'System Failures', 'Fraudulent Activities', 'Regulatory Fines/Penalties', 'Infrastructure Failures', 'Strategic Missteps', 'Reputation Damages', 'Regulatory Changes', 'Global Events Impacting Firms', 'Technological Disruptions']


In [ ]:
df.head()

,Unnamed: 0,title,date,stock
0,0.0,Stocks That Hit 52-Week Highs On Friday,2020-06-05 10:30:00-04:00,A
1,1.0,Stocks That Hit 52-Week Highs On Wednesday,2020-06-03 10:45:00-04:00,A
2,2.0,71 Biggest Movers From Friday,2020-05-26 04:30:00-04:00,A
3,3.0,46 Stocks Moving In Friday's Mid-Day Session,2020-05-22 12:45:00-04:00,A
4,4.0,B of A Securities Maintains Neutral on Agilent...,2020-05-22 11:38:00-04:00,A


# New section